In [1]:
#import python packages
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import pickle

%matplotlib inline

In [2]:
#import EKG analysis packages
import challengeOld as challenge
import waveOld as wave

In [3]:
records = wave.getRecords('~', _not=True)

In [4]:
feat_list = []
for record in records[0]:
    signal = challenge.Signal(record, wave.load(record))
    feat_list.append(challenge.feature_extract(signal))


In [56]:
labels=[]
N = 0;
O = 0;
A = 0;
NO = 0;
for label in records[1]:
    if label == 'N':
        N+=1;
        labels.append(0)
    elif label == 'O':
        O+=1;
        labels.append(1)
    elif label == 'A':
        A+=1;
        labels.append(2)

In [57]:
n_all = len(labels)
print(n_all)

8244


In [58]:
print (NO)

0


In [59]:
class_weights = [N/n_all, O/n_all, A/n_all]
print (class_weights)

[0.6125667151868025, 0.2979136341581756, 0.08951965065502183]


In [60]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(feat_list, labels, test_size=0.20, random_state=42)

In [61]:
from sklearn.preprocessing import scale
X_train = scale(X_train)
X_test = scale(X_test)

In [83]:
from sklearn.decomposition import PCA
pca = PCA(n_components=12)
pca.fit(X_train)

X_train_PCA = pca.transform(X_train)

In [84]:
pca.fit(X_test)
X_test_PCA = pca.transform(X_test)

In [85]:
print (len(X_train_PCA))

6595


In [86]:
EPOCHS = 100
BATCH_SIZE = 128
learning_rate = 0.0001
num_examples = len(X_train)

In [87]:
X = tf.placeholder(tf.float32)
y = tf.placeholder(tf.int32)
keep_prob = tf.placeholder(tf.float32)
one_hot_y = tf.one_hot(y, 3)
class_w = tf.constant(class_weights)

In [107]:
fw1 = tf.Variable(tf.truncated_normal([12, 20], mean=0, stddev=0.1))
fb1 = tf.Variable(tf.zeros([20]))
fw2 = tf.Variable(tf.truncated_normal([20, 20], mean=0, stddev=0.1))
fb2 = tf.Variable(tf.zeros([20]))
fw3 = tf.Variable(tf.truncated_normal([20, 3], mean=0, stddev=0.1))
fb3 = tf.Variable(tf.zeros([3]))


In [108]:
#fully connect layer 1, 58 => 80
fully_layer_1 = tf.add(tf.matmul(X, fw1), fb1)
fully_layer_1 = tf.nn.relu(fully_layer_1)
#fully_layer_1 = tf.nn.dropout(fully_layer_1, keep_prob)

#fully connect layer 2, 80 => 80
fully_layer_2 = tf.add(tf.matmul(fully_layer_1, fw2), fb2)
fully_layer_2 = tf.nn.relu(fully_layer_2)
#fully_layer_2 = tf.nn.dropout(fully_layer_2, keep_prob)

#output layer => 80 => 3
logits = tf.multiply(tf.add(tf.matmul(fully_layer_2, fw3), fb3), class_w)


In [109]:
#train the model
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=one_hot_y, logits=logits)
loss_operation = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(loss_operation)

#accuracy and loss
correct_prediction = tf.equal(tf.argmax(logits,1), tf.argmax(one_hot_y,1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
saver = tf.train.Saver()

In [110]:
def evaluate(X_data, y_data, class_weights):
    num_examples = len(X_data)
    total_accuracy = 0
    sess = tf.get_default_session()
    for offset in range(0, num_examples, BATCH_SIZE):
        batch_X, batch_y = X_data[offset:offset+BATCH_SIZE], y_data[offset:offset+BATCH_SIZE]
        accuracy = sess.run(accuracy_operation, feed_dict={X: batch_X, y: batch_y, class_w: class_weights})
        total_accuracy += (accuracy * len(batch_X))
    return total_accuracy / num_examples

In [111]:
from sklearn.utils import shuffle
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    X_train_PCA, y_train = shuffle(X_train_PCA, y_train)
    for i in range(EPOCHS):
        sess.run(training_op, feed_dict={X: X_train_PCA, y: y_train, class_w: class_weights})
        
        training_accuracy = evaluate(X_train_PCA, y_train, class_weights)
        validation_accuracy = evaluate(X_test_PCA, y_test, class_weights)
        print ('EPOCHS {}..'.format(i+1))
        print ('training_accuracy = {:.3f}'.format(validation_accuracy))
        print ('validation_accuracy = {:.3f}'.format(validation_accuracy))
        print ()
        
    saver.save(sess, 'model')
    print ('model_saved')

EPOCHS 1..
training_accuracy = 0.389
validation_accuracy = 0.389

EPOCHS 2..
training_accuracy = 0.390
validation_accuracy = 0.390

EPOCHS 3..
training_accuracy = 0.393
validation_accuracy = 0.393

EPOCHS 4..
training_accuracy = 0.394
validation_accuracy = 0.394

EPOCHS 5..
training_accuracy = 0.396
validation_accuracy = 0.396

EPOCHS 6..
training_accuracy = 0.398
validation_accuracy = 0.398

EPOCHS 7..
training_accuracy = 0.396
validation_accuracy = 0.396

EPOCHS 8..
training_accuracy = 0.399
validation_accuracy = 0.399

EPOCHS 9..
training_accuracy = 0.399
validation_accuracy = 0.399

EPOCHS 10..
training_accuracy = 0.401
validation_accuracy = 0.401

EPOCHS 11..
training_accuracy = 0.406
validation_accuracy = 0.406

EPOCHS 12..
training_accuracy = 0.409
validation_accuracy = 0.409

EPOCHS 13..
training_accuracy = 0.411
validation_accuracy = 0.411

EPOCHS 14..
training_accuracy = 0.411
validation_accuracy = 0.411

EPOCHS 15..
training_accuracy = 0.415
validation_accuracy = 0.415

EPOC

In [113]:
with tf.Session() as sess:
    
    saver.restore(sess, tf.train.latest_checkpoint('.'))
    predictions = tf.argmax(logits,1)
    print(sess.run(predictions, feed_dict={X: X_test_PCA}))

[1 0 0 ..., 0 0 0]
